In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import lxml
from geopy.geocoders import Nominatim

In [2]:
page = 0 #set page number
properties = [] #create empty list

while page !=30:     #while we are not on the last page
    url = f'https://www.pollardproperty.co.uk/properties-for-sale?start={page}' 
    r = requests.get(url)
    soup = bs(r.text, 'lxml')
    listing = soup.find_all('div',class_=['row-fluid eapow-row0 eapow-overview-row','row-fluid eapow-row1 eapow-overview-row'])
    for i in listing:
        offer = 'N'
        location = i.find('a', class_='eapow-property-header-accent').text.strip()
        price = i.find('span', class_='eapow-overview-price propPrice').text

        bed_icon = i.find('i', class_='flaticon-bed')
        if bed_icon:
            bed_number = bed_icon.find_next_sibling('span').text
        bath_icon = i.find('i', class_='flaticon-bath')
        if bath_icon:
            bath_number = bath_icon.find_next_sibling('span').text

        sofa_icon = i.find('i', class_='flaticon-sofa')
        if sofa_icon:
            sofa_number = sofa_icon.find_next_sibling('span').text

        under_offer = i.find('div',  class_='eapow-bannertopright')
        if under_offer:
            offer = 'Y'

        property = location, price, bed_number, bath_number, sofa_number, offer

        properties.append(property)
        
    page = page + 10
        


In [3]:
df = pd.DataFrame(properties)
df = df.rename(columns={0:'Location',1:'Price',2:'Num. Of Bedrooms',3:'Num. of Bathrooms',4:'Num. of Living Rooms',5:'Under Offer'})

In [4]:
split =  df['Price'].str.split('£',n=1, expand=True)
data2 = split[1]

In [5]:
df['Price'] = data2
df.head()

,Location,Price,Num. Of Bedrooms,Num. of Bathrooms,Num. of Living Rooms,Under Offer
0,"East Gills, Scrabster","330,000",3,2,1,N
1,"Olrig Street, Thurso","310,000",5,2,2,N
2,"Main Street, Castletown","300,000",4,4,3,N
3,"Fairview, Halkirk","275,000",6,5,1,N
4,"Manu House, Castletown, Thurso","230,000",4,2,2,Y


In [6]:
location_split = df['Location'].str.rsplit(',', n=1,expand=True)


In [100]:
location_split
data3 = location_split[1]


In [101]:
df['Area'] = data3
df['Location'] = location_split[0]


In [102]:
geolocator = Nominatim(user_agent="MyApp")

place = []
for area in df['Area']:
    location = geolocator.geocode(area)
    l2 = location.latitude, location.longitude
    place.append(l2)

In [83]:
df_new = pd.DataFrame(place)
df_new
frames = [df, df_new]
final = pd.concat(frames, axis=1)
final.drop('Latitude', inplace=True, axis=1)
final.drop('Longitude', inplace=True, axis=1)

,Location,Price,Num. Of Bedrooms,Num. of Bathrooms,Num. of Living Rooms,Under Offer,Area,0,1
0,East Gills,"330,000",3,2,1,N,Scrabster,58.611019,-3.554907
1,Olrig Street,"310,000",5,2,2,N,Thurso,58.594744,-3.520577
2,Main Street,"300,000",4,4,3,N,Castletown,58.592474,-3.387220
3,Fairview,"275,000",6,5,1,N,Halkirk,58.511676,-3.490252
4,"Manu House, Castletown","230,000",4,2,2,Y,Thurso,58.594744,-3.520577
5,Mey,"220,000",3,2,2,Y,None,44.933143,7.540121
6,Trantlemore,"220,000",3,2,1,N,Forsinard,58.357028,-3.898009
7,Church Lane,"205,000",3,2,2,Y,Halkirk,58.511676,-3.490252
8,9 Strathnaver,"200,000",3,1,1,N,Kinbrace,58.259658,-3.941710
9,Rockwell Terrace,"190,000",4,2,1,Y,Thurso,58.594744,-3.520577


In [91]:
final = final.rename(columns = {0:'Latitude',1:'Longitude'})
final.head()

,Location,Price,Num. Of Bedrooms,Num. of Bathrooms,Num. of Living Rooms,Under Offer,Area,Latitude,Longitude
0,East Gills,"330,000",3,2,1,N,Scrabster,58.611019,-3.554907
1,Olrig Street,"310,000",5,2,2,N,Thurso,58.594744,-3.520577
2,Main Street,"300,000",4,4,3,N,Castletown,58.592474,-3.387220
3,Fairview,"275,000",6,5,1,N,Halkirk,58.511676,-3.490252
4,"Manu House, Castletown","230,000",4,2,2,Y,Thurso,58.594744,-3.520577


In [99]:
import folium
map = folium.Map(location=[final.Latitude.mean(),final.Longitude.mean()], zoom_start=10)

for index, location_info in final.iterrows():
    folium.Marker([location_info["Latitude"], location_info["Longitude"]], popup=location_info["Area"]).add_to(map)

In [98]:
map